In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [9]:
# Read raw CSV files into a dataframe.
movies = pd.read_csv("C:\\Users\\KIIT\\Movie Recommendation System\\tmdb_5000_movies.csv") 
credits =pd.read_csv("C:\\Users\\KIIT\\Movie Recommendation System\\tmdb_5000_credits.csv")

In [10]:
movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [11]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [12]:
movies.isnull().sum()

budget                     0
genres                     0
homepage                3091
id                         0
keywords                   0
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
dtype: int64

In [13]:
movies.describe()

,budget,id,popularity,revenue,runtime,vote_average,vote_count
count,4.803000e+03,4803.000000,4803.000000,4.803000e+03,4801.000000,4803.000000,4803.000000
mean,2.904504e+07,57165.484281,21.492301,8.226064e+07,106.875859,6.092172,690.217989
std,4.072239e+07,88694.614033,31.816650,1.628571e+08,22.611935,1.194612,1234.585891
min,0.000000e+00,5.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,7.900000e+05,9014.500000,4.668070,0.000000e+00,94.000000,5.600000,54.000000
50%,1.500000e+07,14629.000000,12.921594,1.917000e+07,103.000000,6.200000,235.000000
75%,4.000000e+07,58610.500000,28.313505,9.291719e+07,118.000000,6.800000,737.000000
max,3.800000e+08,459488.000000,875.581305,2.787965e+09,338.000000,10.000000,13752.000000


In [14]:
movies = movies.merge(credits,on='title')

In [15]:
movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,206647,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,49026,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,49529,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [16]:
# Drop the unnecessary attributes
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew', 'popularity', 'vote_average']]

In [17]:
movies.head()

,movie_id,title,overview,genres,keywords,cast,crew,popularity,vote_average
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",150.437577,7.2
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...",139.082615,6.9
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...",107.376788,6.3
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...",112.312950,7.6
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...",43.926995,6.1


In [18]:
def convformat(text):
    L = []
    for x in ast.literal_eval(text): 
        L.append(x['name'])
    return L

# Function to extract names of people whose job is 'Director'
def crewdirector(text):
    L = []
    for x in ast.literal_eval(text): 
        if x['job'] == 'Director':  
            L.append(x['name'])
    return L

# Function to remove spaces from each element in a list
def collapse(L):
    return [i.replace(" ", "") for i in L]

# Ensure the 'overview' column is not NaN before applying the split operation
movies.dropna(inplace=True)
movies['overview'] = movies['overview'].astype(str).apply(lambda x: x.split())

In [19]:
def safe_literal_eval(val):
    try:
        return ast.literal_eval(val) if isinstance(val, str) else val
    except:
        return []  # Return an empty list if conversion fails

# Convert necessary columns to lists
for col in ['genres', 'keywords', 'cast', 'crew']:
    movies[col] = movies[col].apply(safe_literal_eval)

# Ensure 'overview' is a list of words
movies['overview'] = movies['overview'].astype(str).apply(lambda x: x.split())

# Merge columns into a single 'tags' column
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [20]:
movies.head()

,movie_id,title,overview,genres,keywords,cast,crew,popularity,vote_average,tags
0,19995,Avatar,"[['In',, 'the',, '22nd',, 'century,',, 'a',, '...",[],[],[],[],150.437577,7.2,"[['In',, 'the',, '22nd',, 'century,',, 'a',, '..."
1,285,Pirates of the Caribbean: At World's End,"[['Captain',, 'Barbossa,',, 'long',, 'believed...",[],[],[],[],139.082615,6.9,"[['Captain',, 'Barbossa,',, 'long',, 'believed..."
2,206647,Spectre,"[['A',, 'cryptic',, 'message',, 'from',, 'Bond...",[],[],[],[],107.376788,6.3,"[['A',, 'cryptic',, 'message',, 'from',, 'Bond..."
3,49026,The Dark Knight Rises,"[['Following',, 'the',, 'death',, 'of',, 'Dist...",[],[],[],[],112.312950,7.6,"[['Following',, 'the',, 'death',, 'of',, 'Dist..."
4,49529,John Carter,"[['John',, 'Carter',, 'is',, 'a',, 'war-weary,...",[],[],[],[],43.926995,6.1,"[['John',, 'Carter',, 'is',, 'a',, 'war-weary,..."


### Stemming Phase
Stemming in Machine Learning (ML) is a Natural Language Processing (NLP) technique used to reduce words to their root or base form. This helps in normalizing text data and improving text-based models like search engines, chatbots, and sentiment analysis.

Example of Stemming : 

Running -> Run

Happily -> Happi

In [22]:
ps = PorterStemmer() 
def stem(text):
  y = []
  for i in text.split(): y.append(ps.stem(i))
  return " ".join(y)

### CountVectorizer
CountVectorizer is a technique from scikit-learn used for converting text data into a bag-of-words (BoW) representation. (CountVectorizer is used to convert text data into a numerical representation (bag-of-words model)). It counts the occurrences of each word in a text document and represents it as a numerical vector, which is useful for machine learning models.

How CountVectorizer Works

-> Tokenization → Splits text into words (tokens).

-> Lowercasing → Converts words to lowercase (default behavior).

-> Stopword Removal (optional) → Removes common words like "the", "is", etc.

-> Vocabulary Creation → Assigns an index to each unique word.

-> Vectorization → Creates a frequency matrix of words across documents.



In [24]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_features=5000, stop_words='english')

# Convert lists to space-separated strings
movies['tags'] = movies['tags'].apply(lambda x: " ".join(x))  #This is necessary because CountVectorizer requires strings, not lists.

# Now apply CountVectorizer
vector = cv.fit_transform(movies['tags']).toarray()


### Cosine-similarity engine
Cosine similarity is a metric used to measure how similar two vectors are, based on the cosine of the angle between them. It is commonly used in text analysis, recommendation systems, and machine learning to compare document similarity, embeddings, and other vector-based representations.

In [26]:
similarity = cosine_similarity(vector)

In [44]:
sorted(list(enumerate(similarity[0])), reverse=True, key=lambda x:x[1])[1:6]

[(151, 0.2773500981126146),
 (3606, 0.2631174057921088),
 (529, 0.17541160386140586),
 (2132, 0.16012815380508716),
 (1214, 0.13355611265883557)]

### Model recommendation Phase

In [49]:
def recommend(movie):
    index = movies[movies['title'] == movie].index[0] 
    distances = similarity[index]
    movies_list = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])[1:6]
    for i in movies_list:
        print(movies.iloc[i[0]].title)

In [51]:
recommend('Avatar')

Beowulf
Apollo 18
Tears of the Sun
The American
Aliens vs Predator: Requiem


In [53]:
movies.head()

,movie_id,title,overview,genres,keywords,cast,crew,popularity,vote_average,tags
0,19995,Avatar,"[['In',, 'the',, '22nd',, 'century,',, 'a',, '...",[],[],[],[],150.437577,7.2,"['In', 'the', '22nd', 'century,', 'a', 'parapl..."
1,285,Pirates of the Caribbean: At World's End,"[['Captain',, 'Barbossa,',, 'long',, 'believed...",[],[],[],[],139.082615,6.9,"['Captain', 'Barbossa,', 'long', 'believed', '..."
2,206647,Spectre,"[['A',, 'cryptic',, 'message',, 'from',, 'Bond...",[],[],[],[],107.376788,6.3,"['A', 'cryptic', 'message', 'from', 'Bond’s', ..."
3,49026,The Dark Knight Rises,"[['Following',, 'the',, 'death',, 'of',, 'Dist...",[],[],[],[],112.312950,7.6,"['Following', 'the', 'death', 'of', 'District'..."
4,49529,John Carter,"[['John',, 'Carter',, 'is',, 'a',, 'war-weary,...",[],[],[],[],43.926995,6.1,"['John', 'Carter', 'is', 'a', 'war-weary,', 'f..."
